In [165]:
%run "..\general_functions\generalFunctions.ipynb"
%run "..\Innovation CBC Slide Duplicate\Innovation CBC Replacement Function.ipynb"

### Parameters

In [166]:
filePath = r"Innovation CBC Datasets/Edgewell Mexico Inno Extract.xlsx"
data_source = "DATA SOURCE: Consumer Test | June 2025"
market='Walmart'
sheet_name1='Performance'
sheet_name2='Ranking'
wdtext="WD assumption = 50 WD"
unitorvolshare= 'New Unit Share' #New Volume Share
client= 'Edgewell'

### Reading Data in Excel

In [167]:


performance = pd.read_excel(filePath,'Performance').rename(columns={'Unnamed: 2': 'Product', 'Unnamed: 3': 'ProductSize'})
rank = pd.read_excel(filePath,'Ranking')
sourcing = pd.read_excel(filePath,"Sourcing")
fair = pd.read_excel(filePath,"Fair")
merged = pd.merge(sourcing, fair, left_on="Rescaled share diff", right_on="Fair Share", how="inner", suffixes=('', '_f'))


### Data Cleaning

In [168]:
numberOfSlides=performance.Grouping.value_counts().reset_index()
numberOfSlides['Count']=numberOfSlides['count'].apply(lambda x : 1 if x<=5 else ((x//5) +( 0 if x%5 ==0 else 1)) )

In [169]:
# Grouping the 'performance' DataFrame by the 'Grouping' column and getting unique 'Sheets names' for each group
columnName = performance.groupby('Grouping')['Sheets names'].unique().reset_index()

# Creating a dictionary where each key is a unique 'Grouping' value and each value is a DataFrame containing 
# 'New volume share' column and the columns listed in 'Sheets names' for that group
dfDic = {columnName['Grouping'].iloc[i]: rank[[unitorvolshare] + list(columnName['Sheets names'].iloc[i])] for i in range(columnName.shape[0])}

# Initializing an empty dictionary to store split DataFrames
dfDicSplit = {}

# Column name to be prioritized in the final DataFrames
col = unitorvolshare

# Iterating over each key-value pair in dfDic
for key, value in dfDic.items():
    # Checking if the number of columns (excluding 'New volume share') is greater than 5
    if value.shape[1] - 1 > 5:
        numOfColumns = 0
        # Splitting the DataFrame into chunks of 5 columns if there are more than 5 columns
        while numOfColumns < value.shape[1] - 1:
            print('split number = ', numOfColumns)
            # Creating a new DataFrame with the next set of 5 columns
            dfDicSplit[key + '_Num ' + str(numOfColumns)] = value.iloc[:, numOfColumns + 1:numOfColumns + 6]
            # Adding the 'New volume share' column to the new DataFrame
            dfDicSplit[key + '_Num ' + str(numOfColumns)][col] = value[col]
            # Reordering columns to place 'New volume share' at the front
            columns_order = [col] + [cl for cl in dfDicSplit[key + '_Num ' + str(numOfColumns)].columns if cl != col]
            dfDicSplit[key + '_Num ' + str(numOfColumns)] = dfDicSplit[key + '_Num ' + str(numOfColumns)].reindex(columns=columns_order)
            numOfColumns += 5
    else:
        # If the number of columns (excluding 'New volume share') is 5 or fewer, add the DataFrame as is to dfDicSplit
        dfDicSplit[key] = value
        dfDicSplit[key]["To color"] = rank["To color"]


In [170]:
# Creating a dictionary to store the count of columns (excluding 'New volume share' and the 'Grouping' column)
# for each DataFrame in dfDicSplit
duplicationList={key:value.shape[1]-2 for key,value in dfDicSplit.items()}
duplicationList

{'Advanced Protection': 1,
 'Baby': 1,
 'Beautycare': 4,
 'Kids': 1,
 'Ozono': 3,
 'Silk Hydration': 3,
 'Sport': 4}

In [171]:
# Grouping the 'performance' DataFrame by the 'Grouping' column and aggregating 'Sheets names' and 'Product' into lists
grouped = performance.groupby('Grouping')[['Sheets names', 'Product']].agg(list)

# Creating a dictionary where each key is a 'Grouping' value and each value is a dictionary 
# with 'Sheets names' as keys and 'Product' as values
result = grouped.apply(lambda x: dict(zip(x['Sheets names'], x['Product'])), axis=1).to_dict()

# Initializing an empty list to store the number of products minus one for each grouping
sum_list = []

# Iterating over each inner dictionary in the result dictionary
for inner_dict in result.values():
    # Calculating the number of products for the current grouping
    inner_sum = len(inner_dict.values())
    # Appending the number of products minus one to sum_list
    sum_list.append(inner_sum - 1)


### Duplication Stage

In [172]:
startIndex=8
index = [list(duplicationList.values()),7,[i+startIndex for i in sum_list]]
duplication = [1,performance.Grouping.nunique(),1]
section_names = ["Innovation ranking","Innovation potential","Innovation sourcing"]

path = os.getcwd() + '//Inno CBC base Oct 2024.pptx'
new_pre = os.getcwd() + '//slide duplicated.pptx'

In [173]:
print(len(index))
print(len(duplication))
print(len(section_names))
print(index)
print(duplication)
print(section_names)


3
3
3
[[1, 1, 4, 1, 3, 3, 4], 7, [8, 8, 11, 8, 10, 10, 11]]
[1, 7, 1]
['Innovation ranking', 'Innovation potential', 'Innovation sourcing']


In [174]:
## Shouldn't be hased in the first run
slideDuplication(index,duplication,section_names,path,new_pre)

In [175]:
prs = Presentation(new_pre)

### Slide 1

In [176]:
numOfDuplicatesFirstSlide = len(list(duplicationList.values())) * duplication[0]
innovationRanking(prs,dfDicSplit,performance,unitorvolshare,client,numOfDuplicates=numOfDuplicatesFirstSlide,position=0)

Advanced Protection
Baby
Beautycare
Kids
Ozono
Silk Hydration
Sport


### Slide 2

In [177]:
positionOfSecondSlide = numOfDuplicatesFirstSlide 
innovationPotential(prs,performance, wdtext,numOfDuplicates = duplication[1],position=positionOfSecondSlide)

5
5
5
5
5
5
5


### Side 3

In [178]:
# numOfDuplicatesThirdSlide = numOfDuplicatesFirstSlide + duplication[1]
numOfDuplicatesThirdSlide = len(list(sum_list)) * duplication[2]
positionOfThirdSlide = positionOfSecondSlide + duplication[1]
innovationSourcing(prs,performance, merged,result, sum_list, numOfDuplicates = numOfDuplicatesThirdSlide, position=positionOfThirdSlide)

                                  Rescaled share diff  1_AP360_370  \
8     Hawaiian Tropic Island Sport Lotion FPS50 240ml    -0.085390   
11    Hawaiian Tropic Sheer Touch Lotion FPS50+ 240ml    -0.079383   
64  Nivea Sun Protection & Hydration Cream FPS50+ ...    -0.075745   
3           Hawaiian Tropic Ozono Lotion FPS50+ 240ml    -0.062523   
34  Banana Boat Advanced Protection Lotion FPS50+ ...    -0.052141   
55  Nivea Sun Protect & Refresh Sport Lotion FPS50...    -0.051530   
53  Nivea Sun Protection Sensitive Skin Gel FPS50+...    -0.050487   
29        Banana Boat Aqua Protect Cream FPS50+ 236ml    -0.046932   
25  Banana Boat Aqua Protect Sport Lotion FPS50+ 2...    -0.036718   
54  Nivea Sun Protect & Refresh Sport Spray FPS50 ...    -0.028104   
65  Nivea Sun Protection & Hydration Lotion FPS50+...    -0.026681   

    1_AP360_370_f       sum  
8        2.394444 -0.085390  
11       1.822830 -0.079383  
64       0.800369 -0.075745  
3        1.433749 -0.062523  
34       

In [194]:
from datetime import date
today = str(date.today())
outputPath=os.getcwd() + "\\Innovation CBC "+market+" output "+today+".pptx"
prs.save(outputPath)
app = win32.Dispatch("PowerPoint.Application")
presentation = app.Presentations.Open(outputPath)

In [195]:
ranking_slide_count = max(duplication)
potential_slide_count = max(duplication) 
sourcing_slide_count = max(duplication)

print(f"Slide counts - Ranking: {ranking_slide_count}, Potential: {potential_slide_count}, Sourcing: {sourcing_slide_count}")


app = win32.Dispatch("PowerPoint.Application")
presentation = app.Presentations.Open(outputPath)

new_order = []
max_slides_per_section = max(ranking_slide_count, potential_slide_count, sourcing_slide_count)

for round_num in range(max_slides_per_section):
    if round_num < ranking_slide_count:
        ranking_slide_index = round_num + 1  
        new_order.append(ranking_slide_index)
        
    if round_num < potential_slide_count:
        potential_slide_index = ranking_slide_count + round_num + 1
        new_order.append(potential_slide_index)
        
    if round_num < sourcing_slide_count:
        sourcing_slide_index = ranking_slide_count + potential_slide_count + round_num + 1
        new_order.append(sourcing_slide_index)

print(f"New slide order (1-based): {new_order}")

def reorder_slides_in_place(presentation, new_order):
    total_slides = presentation.Slides.Count
    
    slides_temp = []
    
    for slide_index in new_order:
        if slide_index <= total_slides:
            original_slide = presentation.Slides(slide_index)
            original_slide.Copy()
            time.sleep(0.5) 
            new_slide = presentation.Slides.Paste(presentation.Slides.Count + 1)
            slides_temp.append(new_slide)
    
    for i in range(total_slides, 0, -1):
        presentation.Slides(i).Delete()
    
    print(f"Reordered {len(slides_temp)} slides successfully")
    return presentation

# Reorder the slides
presentation = reorder_slides_in_place(presentation, new_order)

reordered_outputPath = os.getcwd() + "\\Innovation CBC "+market+" reordered output "+today+".pptx"
def remove_all_sections(presentation):
    try:
        section_count = presentation.SectionProperties.Count
        print(f"Removing {section_count} sections...")
        for i in range(section_count, 0, -1):
            presentation.SectionProperties.Delete(i, False)  # False = keep slides
        print("All sections removed.")
        return presentation
    except Exception as e:
        print(f"Error removing sections: {e}")

presentation = remove_all_sections(presentation)
presentation.SaveAs(reordered_outputPath)

print(f"Reordered presentation saved as: {reordered_outputPath}")
print("Slide reordering complete! All formatting preserved.")

Slide counts - Ranking: 7, Potential: 7, Sourcing: 7
New slide order (1-based): [1, 8, 15, 2, 9, 16, 3, 10, 17, 4, 11, 18, 5, 12, 19, 6, 13, 20, 7, 14, 21]
Reordered 21 slides successfully
Removing 3 sections...
All sections removed.
Reordered presentation saved as: c:\Users\SophieZIMMERMANN\Documents\Slide-Automate\Innovation CBC Slide Duplicate\Innovation CBC Walmart reordered output 2025-07-24.pptx
Slide reordering complete! All formatting preserved.
